In [ ]:
import os
import sys
from dotenv import load_dotenv
# Step 1 — Go up one level (from /notebooks/bank_reviews → project_root)
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)
# In your Notebook cell:
from medical_warehouse.Scripts.yolo_data_loader import YoloDataHandler

from medical_warehouse.Scripts.yolo_detect import YOLOAnalyzer
from medical_warehouse.Scripts.config import settings


In [6]:

# 1. Initialize the analyzer
# Model name is pulled from a constant or passed explicitly
analyzer = YOLOAnalyzer(model_name='yolov8n.pt')

# 2. Run detection using centralized config paths
# This replaces the hardcoded "../data/raw/images"
image_folder = os.path.join(
    settings.PROJECT.BASE_DATA_DIR, 
    settings.PROJECT.IMAGE_SUBDIR
)

print(f"🚀 Scanning images in: {image_folder}")
detections_df = analyzer.detect_objects(image_folder)

# 3. Save and Display
if detections_df is not None:
    # Use config for the output path as well
    output_path = os.path.join(settings.PROJECT.BASE_DATA_DIR, "image_detections.csv")
    analyzer.save_results(detections_df, output_path)
    
    # Show the first 5 rows
    display(detections_df.head())
    
    # Quick Summary using the dynamic dataframe
    print("\n--- Image Category Summary ---")
    summary = detections_df['image_category'].value_counts()
    print(summary)
else:
    print("❌ No images were processed. Check your path and file extensions.")

2026-02-17 15:47:25,835 - INFO - YOLO model yolov8n.pt initialized.
2026-02-17 15:47:25,899 - INFO - 🔍 Starting detection on 2292 images...


🚀 Scanning images in: c:\Users\Birhanu Matebe\Downloads\KAIM\Project\Ethiopian-Medical-Bussines\Ethiopian-Medical-Business-Intelligence\data\raw/images


2026-02-17 15:55:47,964 - INFO - ✅ Detection results saved to: c:\Users\Birhanu Matebe\Downloads\KAIM\Project\Ethiopian-Medical-Bussines\Ethiopian-Medical-Business-Intelligence\data\image_detections.csv


,message_id,detected_objects,confidence_score,image_category,image_path
0,10,none,0.0000,other,c:\Users\Birhanu Matebe\Downloads\KAIM\Project...
1,11,clock,0.4273,other,c:\Users\Birhanu Matebe\Downloads\KAIM\Project...
2,13,"hot dog, donut, hot dog, donut, hot dog, hot dog",0.5039,other,c:\Users\Birhanu Matebe\Downloads\KAIM\Project...
3,14,none,0.0000,other,c:\Users\Birhanu Matebe\Downloads\KAIM\Project...
4,15,none,0.0000,other,c:\Users\Birhanu Matebe\Downloads\KAIM\Project...



--- Image Category Summary ---
image_category
other              1293
product_display     756
lifestyle           211
promotional          32
Name: count, dtype: int64


In [3]:
# Initialize the handler
handler = YoloDataHandler()

# Path to your YOLO results
my_csv_path = "../data/image_detections.csv"

# Upload to Postgres
handler.upload_yolo_csv(csv_path=my_csv_path,table_name='image_analysis', schema='processed')

Connected to database: medical_warehouse
Ensured schema 'processed' exists.
Successfully uploaded 1547 rows to processed.image_analysis
